In [3]:
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import mean
from collections import Counter

In [4]:
class PSTool:
    def pyspark_session(self, host_location):
        """
        Creates and returns spark session object
        """
        sc = SparkContext(host_location)  # Create spark context
        spark = SparkSession(sc)  # Create session
        return spark

    def file_loader(self, path, delim, spark_obj):
        data = spark_obj.read.options(delimiter=delim).csv(path)  # load file
        data = data
        return data


if __name__ == "__main__":
    # Instanciate object
    pstool = PSTool()
    # start session
    spk = pstool.pyspark_session('local')
    # load data
    path = 'all_bacilli_subset.tsv'
    df=pstool.file_loader(path, '\t', spk)

In [5]:
# 'Protein _accession', 'Sequence_MD5_digest', 'Sequence_length', 'Analysis',
# 'Signature_accession', 'Signature_description', 'Start location', 'Stop location',
# 'Score', 'Status', 'Date', 'InterPro annotations', 'InterPro annotations' - description (e.g. BRCA2 repeat)
#     (GO annotations (e.g. GO:0005515) - optional column; only displayed if –goterms option is switched on)
#     (Pathways annotations (e.g. REACT_71) - optional column; only displayed if –pathways option is switched on)

The TSV format presents the match data in columns as follows:

    1. Protein accession (e.g. P51587)
    2. Sequence MD5 digest (e.g. 14086411a2cdf1c4cba63020e1622579)
    3. Sequence length (e.g. 3418)
    4. Analysis (e.g. Pfam / PRINTS / Gene3D)
    5. Signature accession (e.g. PF09103 / G3DSA:2.40.50.140)
    6. Signature description (e.g. BRCA2 repeat profile)
    7. Start location
    8. Stop location
    9. Score - is the e-value (or score) of the match reported by member database method (e.g. 3.1E-52)
    10. Status - is the status of the match (T: true)
    11. Date - is the date of the run
    12. InterPro annotations - accession (e.g. IPR002093)
    13. InterPro annotations - description (e.g. BRCA2 repeat)
    14. (GO annotations (e.g. GO:0005515) - optional column; only displayed if –goterms option is switched on)
    15. (Pathways annotations (e.g. REACT_71)-optional column; only displayed if –pathways option is switched on)


# Question 1
How many distinct protein annotations are found in the dataset? I.e. how many distinc InterPRO numbers are there?

In [10]:
df.select("_c11").agg("_c11")
# print(InterPRO_distinc)

AssertionError: all exprs should be Column

# Question 2
How many annotations does a protein have on average?

In [5]:
total_annot = df.select("_c11").count()
total_annot

200

In [6]:
avg_annot = total_annot/InterPRO_distinc
print(avg_annot)

3.278688524590164


# Question 3
What is the most common GO Term found?

In [7]:
# df.groupBy("_c13").count().show()

In [8]:
go_list = []
for i in df.filter(df._c13 != '-').select('_c13').collect():
    go_list.extend(i[0].split("|"))

In [9]:
most_common = max(go_list, key = go_list.count)
most_common

'GO:0004743'

# Question 4
What is the average size of an InterPRO feature found in the dataset?

In [10]:
# df.select("_c2").show()

In [11]:
from pyspark.sql.functions import abs

df1 = df.withColumn('diff', (df._c8 - df._c7))
df1 = df1.withColumn('diff', abs(df1.diff))
# df2.select('abs').show()

In [12]:
mean_size = df1.select(mean('diff')).collect()
mean_size_float = float(str(mean_size[0]).replace('Row(avg(diff)=', '').replace(')', ''))
mean_size_float

275.50452025394884

# Question 5
What is the top 10 most common InterPRO features?

In [13]:
def get_n_most_comm(df, colname, n):
    interPRO_feat = []
    for i in df.filter(df[colname] != '-').select(colname).collect():
        interPRO_feat.extend(i)
    
    c = Counter(interPRO_feat)
    return c.most_common(n)
    
    
most_common_feats = get_n_most_comm(df, '_c11', 10)
most_common_feats

[('IPR001697', 10),
 ('IPR005814', 6),
 ('IPR000456', 5),
 ('IPR001867', 4),
 ('IPR004358', 4),
 ('IPR003776', 3),
 ('IPR001789', 3),
 ('IPR000551', 3),
 ('IPR000515', 3),
 ('IPR036890', 3)]

# Question 6
If you select InterPRO features that are almost the same size (within 90-100%) as the protein itself, what is the top10 then?

In [14]:
def filter90(df):
    print('Initial shape', df.count(), len(df.columns))
    # df1.select('diff').show()
    dfx = df.withColumn('rel_size', (df.diff / df._c2))
    dfx = dfx.filter(dfx.rel_size >= 0.9)
    print('End shape', dfx.count(), len(dfx.columns))
    return dfx

df_filt90 = filter90(df1)
most_common_feats_large = [i[0] for i in get_n_most_comm(df_filt90, '_c11', 10)]
most_common_feats_large 

Initial shape 200 16
End shape 78 17


['IPR001867',
 'IPR005814',
 'IPR000456',
 'IPR036890',
 'IPR003594',
 'IPR004358',
 'IPR003776',
 'IPR036244',
 'IPR036373',
 'IPR000515']

# Question 7
If you look at those features which also have textual annotation, what is the top 10 most common word found in that annotation?
https://stackoverflow.com/questions/65808504/spark-find-most-common-words-in-dataset

In [15]:
# df.select('_c12').show()
def get_most_common_words(df, column, sep, n):
    annot_list = []
    for i in df.filter(df[column] != '-').select(column).collect():
        annot_list.extend(i[0].split(sep))

    c = Counter(annot_list)
    return c.most_common(n)


top10words = [i[0] for i in get_most_common_words(df, '_c12', " ", 10)]

print(top10words)

['domain', 'superfamily', 'Pyruvate', 'protein', 'kinase', 'kinase,', 'C-terminal', 'Signal', 'transduction', 'Ribosomal']


# Question 8
And the top 10 least common?

In [26]:
def get_least_common_words(df, column, sep, n):
    annot_list = []
    for i in df.filter(df[column] != '-').select(column).collect():
        annot_list.extend(i[0].split(sep))

    c = Counter(annot_list)
    return sorted(c.items(), key=lambda x: x[1])[:n]


bottom10words = [i[0] for i in get_least_common_words(df, '_c12', " ", 10)]
print(bottom10words)
df.explain()

['Bacteriocin', 'biosynthesis,', 'cyclodehydratase', 'Thiazole/oxazole-forming', 'peptide', 'maturase,', 'SagD', 'component', 'Transcriptional', 'regulatory']
== Physical Plan ==
FileScan csv [_c0#17,_c1#18,_c2#19,_c3#20,_c4#21,_c5#22,_c6#23,_c7#24,_c8#25,_c9#26,_c10#27,_c11#28,_c12#29,_c13#30,_c14#31] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/C:/Users/Martin Schepers/Documents/GitHub/programming3/Assignmen..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<_c0:string,_c1:string,_c2:string,_c3:string,_c4:string,_c5:string,_c6:string,_c7:string,_c...




# Question 9
Combining your answers for Q6 and Q7, what are the 10 most commons words found for the largest InterPRO features?

In [25]:
top10words_large = [i[0] for i in get_most_common_words(df_filt90, '_c12', " ", 10)]
top10words_large

['domain',
 'superfamily',
 'protein',
 'Histidine',
 'kinase/HSP90-like',
 'ATPase',
 'Ribosomal',
 'L17',
 'DNA-binding',
 'MetI-like']

# Question 10
What is the coefficient of correlation ( 𝑅2 ) between the size of the protein and the number of features found?